In [3]:
import cv2

first_frame = None
video = cv2.VideoCapture(0)
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:

    _, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    fgmask = fgbg.apply(frame)
    fgmask = cv2.Canny(fgmask, 1,1)

    if first_frame is None:
        first_frame = gray
        continue

    delta_frame = cv2.absdiff(first_frame, gray)
    th_delta = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    th_delta = cv2.dilate(th_delta, None , iterations=1)
    
    cnts, _ = cv2.findContours(th_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    text = 'Object'
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1
    color = (0, 255, 0)
    thickness = 1
    
    for contour in cnts:
        if cv2.contourArea(contour) < 1000 :
            continue
        (x,y,w,h) = cv2.boundingRect(contour)
        position = (x, y)
        cv2.putText(frame, text, position, font, scale, color, thickness, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.fillPoly(frame, pts=cnts, color=cv2.mean(frame,fgmask))
        cv2.drawContours(frame, contour, -1, (52, 116, 172), 3)
   
    cv2.imshow('Capturing', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

video.release()
cv2.destroyAllWindows()
